In [1]:
import pandas as pd
import os

from passlib.context import CryptContext

import sqlite3, sqlalchemy
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData, create_engine, text, inspect, Boolean, DateTime, Float
from IPython.display import Markdown, display

In [2]:
df = pd.read_csv('../data/dataset.csv', index_col=0)

In [4]:
df.tail()

,user_id,time_stamp,artist_id,artist_name,track_id,track_name
53,1005,2022-01-27 21:43:14,1902,Jay-Z,12269,Show Me What You Got
54,1005,2022-01-27 21:43:14,1902,Jay-Z,38806,Politics As Usual
55,1005,2022-01-27 21:43:14,1902,Jay-Z,38994,"Girls, Girls, Girls"
56,1005,2022-01-27 21:43:14,1902,Jay-Z,39526,Justify My Thug
57,1005,2022-01-27 21:43:14,1902,Jay-Z,39550,Hard Knock Life


## Create Database

In [36]:
mysql_url = 'localhost'  # to complete
mysql_user = 'root'
mysql_password = 'pytune'  # to complete
database_name = 'main'

# recreating the URL connection
connection_url = 'mysql://{user}:{password}@{url}/{database}'.format(
    user=mysql_user,
    password=mysql_password,
    url=mysql_url,
    database=database_name
)

In [37]:
engine = create_engine(connection_url)

In [9]:
meta = MetaData()

## users

In [11]:
user = Table(
    'user', meta, 
    Column('id', Integer, primary_key = True, autoincrement=False),
    Column('name', String(32), unique=True),
    Column('admin', Boolean),
    Column('hashed_password', String(128)),
    extend_existing=True,
)
meta.create_all(engine)

In [12]:

pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

In [13]:
values = [(0,'admin',True, pwd_context.hash('admin')),
          (960, 'user_jazz', False, pwd_context.hash('jazz')),
          (961, 'user_classic', False, pwd_context.hash('classic')),
          (962, 'user_pop', False, pwd_context.hash('pop')),
          (963, 'user_rock', False, pwd_context.hash('rock')),
          (964, 'user_rap', False, pwd_context.hash('rap'))]

pw = pwd_context.hash(f'user')

for i in range(959):
    values.append((i+1, f'user_{i+1}', False, pw))


In [14]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            ins = f"INSERT INTO user VALUES (%s,%s,%s,%s)"
            connection.execute(ins, values)
        except Exception as e:
            print(e)
            transaction.rollback()
        else:
            transaction.commit()

In [15]:
with engine.connect() as connection:
    result = connection.execute("Select * from user limit 10;")
    print(result.fetchall())

[(0, 'admin', 1, '$2b$12$W2MxOJK98GhLuaTkLG6FBurMF.4/QpPUNFm8RFEpbImf5eYG9LQMq'), (1, 'user_1', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (2, 'user_2', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (3, 'user_3', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (4, 'user_4', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (5, 'user_5', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (6, 'user_6', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (7, 'user_7', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (8, 'user_8', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re'), (9, 'user_9', 0, '$2b$12$RU6i5525yAfXEg8asXPJxuBuitHprNP3i6OLdMpq6oJxcFDYgG5re')]


### artist

In [16]:
artist = Table(
    'artist', meta, 
    Column('id', Integer, primary_key = True, autoincrement=False),
    Column('name', String(128)),
    extend_existing=True,
)
meta.create_all(engine)

In [17]:
df_artist = df.groupby(['artist_id']).first().reset_index()[['artist_id','artist_name']]

In [18]:
values = [tuple(el) for el in df_artist.values]

In [19]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            ins = f"INSERT INTO artist VALUES (%s,%s)"
            connection.execute(ins, values)
        except Exception as e:
            print(e)
            transaction.rollback()
        else:
            transaction.commit()

In [20]:
with engine.connect() as connection:
    result = connection.execute("Select * from artist limit 10;")
    print(result.fetchall())

[(0, 'Deep Dish'), (1, '坂本龍一'), (2, 'Underworld'), (3, 'Ennio Morricone'), (4, 'Minus 8'), (5, 'Beanfield'), (7, 'Alif Tree'), (14, '4Hero'), (17, 'Lisa Shaw'), (18, 'Portishead & Moloko')]


### Tracks

In [21]:
track = Table(
    'track', meta, 
    Column('id', Integer, primary_key = True , autoincrement=False),
    Column('title', String(246)),
    Column('artist_id', Integer, ForeignKey('artist.id')),
    Column('artist_name', String(128)),
    extend_existing=True,
)
meta.create_all(engine)

In [22]:
df_track = df.groupby(['track_id']).first().reset_index()[['track_id','track_name','artist_id','artist_name']]

In [23]:
df_track.head()

,track_id,track_name,artist_id,artist_name
0,0,Elysian Fields,4,Minus 8
1,1,Planetary Deadlock,5,Beanfield
2,2,Deadly Species,7,Alif Tree
3,3,Cold Fusion,4,Minus 8
4,4,Look Inside,14,4Hero


In [24]:
values = [tuple(el) for el in df_track.values]

In [25]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            ins = f"INSERT INTO track VALUES (%s,%s,%s,%s)"
            connection.execute(ins, values)
        except Exception as e:
            print(e)
            transaction.rollback()
        else:
            transaction.commit()

In [26]:
with engine.connect() as connection:
    result = connection.execute("Select * from track limit 10;")
    print(result.fetchall())

[(0, 'Elysian Fields', 4, 'Minus 8'), (1, 'Planetary Deadlock', 5, 'Beanfield'), (2, 'Deadly Species', 7, 'Alif Tree'), (3, 'Cold Fusion', 4, 'Minus 8'), (4, 'Look Inside', 14, '4Hero'), (5, 'Morning Child', 14, '4Hero'), (6, 'Dedication To The Horse', 14, '4Hero'), (7, "Why Don'T You Talk?", 14, '4Hero'), (8, 'Sophia', 14, '4Hero'), (9, 'I Feel Blue', 7, 'Alif Tree')]


### User Items

In [27]:
user_item = Table(
    'user_item', meta, 
    Column('user_id', Integer, ForeignKey('user.id')),
    Column('track_id', Integer, ForeignKey('track.id')),
    Column('time_stamp', DateTime),
    extend_existing=True,
)
meta.create_all(engine)

In [28]:
df_user_item = df[['user_id','track_id','time_stamp']]

In [29]:
values = [tuple(el) for el in df_user_item.values][::-1]

In [30]:
with engine.connect() as connection:
    with connection.begin() as transaction:
        try:
            ins = f"INSERT INTO user_item VALUES (%s,%s,%s)"
            connection.execute(ins, values)
        except Exception as e:
            print(e)
            transaction.rollback()
        else:
            transaction.commit()

In [31]:
with engine.connect() as connection:
    result = connection.execute("Select * from user_item limit 10;")
    print(result.fetchall())

[(964, 39550, datetime.date(2022, 1, 27)), (964, 39526, datetime.date(2022, 1, 27)), (964, 38994, datetime.date(2022, 1, 27)), (964, 38806, datetime.date(2022, 1, 27)), (964, 12269, datetime.date(2022, 1, 27)), (964, 77528, datetime.date(2022, 1, 27)), (964, 74054, datetime.date(2022, 1, 27)), (964, 77508, datetime.date(2022, 1, 27)), (964, 72134, datetime.date(2022, 1, 27)), (964, 6283, datetime.date(2022, 1, 27))]


### Predictions

In [32]:
prediction = Table(
    'prediction', meta, 
    Column('user_id', Integer, ForeignKey('user.id')),
    Column('track_ids', String(1024)),
    Column('time_stamp', Date),
    extend_existing=True,
)
meta.create_all(engine)

### Training result

In [33]:
training = Table(
    'training', meta,
    Column('time_stamp', Date),
    Column('sample', Integer),
    Column('precision', Float),
    Column('map', Float),
    Column('ndcg', Float),
    Column('auc', Float),
    extend_existing=True,
)
meta.create_all(engine)